## 1st Notebook

#### Justifications for Data Preparation

This first part of this notebook takes the relation file, finds all the ChEBI ids of compounds with at least one "has_role" relation, extracts them all, and computes some statistics.

It then, more importantly writes a file with roles only. 

This is done to query ChEBI ontology and a file with ChEBI ids, inchis, and roles.

This is done to then query PubChem, and then join this with the previous query.

I only want chemical compounds with 1 role because otherwise, each class will be a combination of roles.
Even if a class only has one role, this will be a combination of roles where one is true and the others are false.
I would like to avoid this so that I may give a clearer picture of the incoming unknown compounds, as opposed to a combination of potential roles. I want one clear direction to go in, as this will speed up drug discovery processes. 

For better training, I would also like the assumption that each role name is directly associated with exactly 1 role type. There is a hierarchy here. Each role type has many different types of role names. 

#### Download Instructions

You will need the 'relation_3star.tsv' and the 'chebiId_inchi_3star.tsv' (or the non-3star versions) files from: https://ftp.ebi.ac.uk/pub/databases/chebi/Flat_file_tab_delimited/

These are the only 2 input files needed to start the entire process. 

This notebook assumes the input is in the data folder. If that's not the case, please change the location of the file in the read_csv line here below.

In [39]:
import requests
import pandas as pd
import sys
import re

# load relations tsv into pandas dataframe
df=pd.read_csv('../data/relation_3star.tsv', sep='\t')

This dataframe contains all the relationships of the compounds. 

We are specifically looking for whether a compound ChEBI ID is correlated to role ChEBI ID. 
ChEBI is an ontology, and even roles have a specific ChEBI ID.

If there is a 'has_role' relationship, FINAL_ID is the compound CheEBI ID, and INIT_ID has the ChEBI ID of the role. 

In [40]:
df

,ID,TYPE,INIT_ID,FINAL_ID,STATUS
0,3,is_a,24431,23367,C
1,18,is_a,23855,22663,E
2,19,is_a,23855,23315,E
3,20,is_a,23855,23514,C
4,22,is_a,23855,24322,E
...,...,...,...,...,...
189086,1128803,is_a,85281,134165,C
189087,1128804,is_a,15897,134156,C
189088,1128805,is_a,85282,134156,C
189089,1128806,is_a,15897,134157,C


In [41]:
print(df.columns)

Index(['ID', 'TYPE', 'INIT_ID', 'FINAL_ID', 'STATUS'], dtype='object')


In [46]:
# this dictionary will contain the number of roles for each ChEBI id, found in "FINAL_ID" in the relationship
chebi_nroles={}

# iterating through dataframe
for i in range(len(df)):
    
    # if we find a row with "has_role" as a relation, then FINAL_ID is the ID of a ChEBI with an INIT_ID role
    if(df.TYPE[i]=='has_role'):
        
        # counting the number of roles this ChEBI id has, for statistical input
        if(df.FINAL_ID[i]) in chebi_nroles:
            
            chebi_nroles[df.FINAL_ID[i]]+=1
            
        else:
            
            chebi_nroles[df.FINAL_ID[i]]=1

In [47]:
# Computing the distribution
# how many ids have nroles?
nroles_nids={}

for i in chebi_nroles:
    
    # if it exists, we increment 
    if(chebi_nroles[i] in nroles_nids):
        
        nroles_nids[chebi_nroles[i]]+=1
        
    # else we create it    
    else:
        
        nroles_nids[chebi_nroles[i]]=1
        
print(nroles_nids)

{6: 223, 1: 11861, 4: 978, 2: 5155, 8: 68, 3: 2269, 13: 7, 7: 147, 5: 471, 10: 26, 18: 1, 9: 38, 12: 11, 11: 16, 19: 1, 25: 1, 20: 1, 16: 4, 15: 1, 23: 1, 14: 1}


Let's print the distribution of number of roles per ids

In [48]:
# Let's sort this by number of roles (which is the "key" of the dictionary) 
# This is x[0] in our lambda.

# output is in the format of "x : y" where x is the number of roles, and y is the size of the population having x as number of roles

for key, value in sorted(nroles_nids.items(), key=lambda x: x[0]): 

    print("{} : {}".format(key, value))

1 : 11861
2 : 5155
3 : 2269
4 : 978
5 : 471
6 : 223
7 : 147
8 : 68
9 : 38
10 : 26
11 : 16
12 : 11
13 : 7
14 : 1
15 : 1
16 : 4
18 : 1
19 : 1
20 : 1
23 : 1
25 : 1


In [27]:
# load the chebiId_inchi file
cinchi=pd.read_csv('../data/chebiId_inchi_3star.tsv', sep='\t')

# building a dictionary out of it, for faster access
chebiid_inchi={}

# iterating through the dataframe and build the dictionary line by line
for i in range(len(cinchi)):
    
    chebiid_inchi[cinchi.CHEBI_ID[i]]=cinchi.InChI[i]    

This dataframe contains the specific InChI key associated with a compound's ChEBI ID.

We need this to query PubChem via the InChi key. PubChem APIs cannot be queried using ChEBI ID.

In [28]:
cinchi

,CHEBI_ID,InChI
0,90,InChI=1S/C15H14O6/c16-8-4-11(18)9-6-13(20)15(2...
1,165,"InChI=1S/C10H16O/c1-9(2)7-4-5-10(3,6-7)8(9)11/..."
2,776,InChI=1S/C18H22O3/c1-18-7-6-13-12-5-3-11(19)8-...
3,943,InChI=1S/C7H3Cl2N/c8-6-2-1-3-7(9)5(6)4-10/h1-3H
4,1148,"InChI=1S/C4H8O3/c1-2-3(5)4(6)7/h3,5H,2H2,1H3,(..."
...,...,...
43606,691037,InChI=1S/C22H29FO4/c1-12-8-16-15-5-4-13-9-14(2...
43607,691622,InChI=1S/C8H10N4O3/c1-10-4-5(9-7(10)14)11(2)8(...
43608,724125,"InChI=1S/C6H11NO3/c1-10-6(9)3-2-5(8)4-7/h2-4,7..."
43609,741548,"InChI=1S/C5H8O4/c1-2-3(4(6)7)5(8)9/h3H,2H2,1H3..."


Creating a convenience file on the side, linking InCHI and role's ChEBI ID to the compound's ChEBI ID.

At the same time, creating a set of all the roles found in our chosen compounds.

In [29]:
# building a set of roles, so we can use them to scrape ChEBI to get their meaning
roles=set()

# preparing the chebiid_role file
fout=open('chebiId_inchi_role.tsv','w')
fout.write("ChEBI\tInChI\tRole\n")

# iterating through dataframe once again, this time only to grab the ids with 1 role only
for i in range(len(df)):
    
    # if we find a row with "has_role" as a relation, then FINAL_ID is the id of a ChEBI with a INIT_ID role
    if(df.TYPE[i]=='has_role'):
        
        # take only the ids with exactly 1 role
        if(chebi_nroles[df.FINAL_ID[i]]==1):
            
            # proceed only if we have an inchi, otherwise we can't query PubChem
            if(df.FINAL_ID[i] in chebiid_inchi):
                
                # roles is a set. The element is only added if not already present.
                roles.add(df.INIT_ID[i])
                
                # writing the chebi id, inchi, and role
                # we will need this to query PubChem
                fout.write(str(df.FINAL_ID[i])+"\t"+str(chebiid_inchi[df.FINAL_ID[i]])+"\t"+str(df.INIT_ID[i])+"\n")

fout.close()

Creating a convenience file on the side, listing all the possible roles for our chosen compounds.

In [30]:
# preparing the role file, we will need this to query ChEBI for role meaning and type
fout=open('roles','w+')

# write the roles to a file, one per line
for i in sorted(roles):
    
    fout.write(str(i)+"\n")
    
fout.close()

### Scraping ChEBI

In ChEBI, I used the ontology to find which role type (biological, chemical, application) these compounds have, as well as the name of that specific role. 

For example, xenobiotic metabolite is the role name, and biological role is the role type.

Specifically, I used the Tree View to do so.

In [7]:
# url for ChEBI webpage. 
# Not using an API, but rather scraping the html page
url_head="https://www.ebi.ac.uk/chebi/chebiOntology.do?chebiId=CHEBI:"
url_tail="&treeView=true"

Because we have previously constructed the 'roles' file to hold all the distinct ChEBI IDs for roles, we can use this to query PubChem.

This way, we can only scrape 603 pages, instead of 1 page per compound ID.

Feel free to run the below code. 

If you would like to skip this step and access the file which it produces directly, you can find it under:

'role_name_type.tsv'

Otherwise, it is loaded 3 cells below, under PubChem API, into a pandas dataframe.

In [8]:
# preparing an output file
fout=open('role_name_type.tsv','w')
fout.write("Role\tName\tType\n")

# cycling through the input file, preparing the request, scraping ChEBI, and writing each result to a file
count=0

with open("roles", "r") as f:
    
  for line in f:
    
    # removing line feed
    line = line.strip()
    
    # making the request
    res=requests.get(url_head+line+url_tail)
    
    role=""
    rtype=""
    
    # splitting output into lines
    for l in res.text.splitlines():
        
        # this line searches for the ontology definitions in the html
        if re.search("<font class=\"checked", l):
            
            # removing html tags
            nl=re.sub('<.*?>', '', l)
            
            # finding out which role is in the html
            if (nl.find("CHEBI:51086 chemical role")>-1):
                
                rtype="chemical role"
                
            else: 
                
                if (nl.find("CHEBI:24432 biological role")>-1):
                    
                    rtype="biological role"
                    
                else:
                    
                    if (nl.find("CHEBI:33232 application")>-1):
                        
                        rtype="application"
                        
            # finding role description (name), which is found starting with CHEBI: and contains a space. 
            if re.search("CHEBI:"+line,nl):
                
                # get everything after the space, which is the role description
                role=nl[nl.index(" ")+1:]
                
    # writing output, tab separated, thus creating a tsv
    fout.write(line+"\t"+role+"\t"+rtype+"\n")
    
    # this line just flushes the output, so we can watch the file grow line by line
    fout.flush()

    # watching the counter grow
    count+=1
    print(str(count), end='\r')
    sys.stdout.flush()
    
fout.close()

### PubChem API

To construct the URL below with the specific query, refer to the following PUG REST Tutorial on RESTful documentation.

https://pubchemdocs.ncbi.nlm.nih.gov/pug-rest-tutorial

URL for RESTFul service from PubChem, already customized for our variables.

When quering by inchi specifically, instead of including the full string of the inchi parameter in the URL, let's pass the inchi as a parameter of the requests.post method. 

The syntax of inchi conflicts the syntax of url. To avoid that, it is necessary to include the inchi as the data of requests.post.

In [33]:
url="https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/inchi/property/MolecularFormula,MolecularWeight,XLogP,TPSA,Complexity,HBondDonorCount,HBondAcceptorCount,RotatableBondCount,HeavyAtomCount,IsotopeAtomCount,ExactMass,MonoisotopicMass,Charge,AtomStereoCount,DefinedAtomStereoCount,UndefinedAtomStereoCount,BondStereoCount/JSON"

In [34]:
# load chebi, inchi, roles into dataframe
df=pd.read_csv('chebiId_inchi_role.tsv', sep='\t')
df.columns

Index(['ChEBI', 'InChI', 'Role'], dtype='object')

This is the first convenience file we created.

In [35]:
df

,ChEBI,InChI,Role
0,19032,InChI=1S/C2H4BrCl/c3-1-2-4/h1-2H2,25435
1,21182,InChI=1S/C23H28Cl3N3O.2ClH/c1-16(4-3-11-29(12-...,24853
2,21183,InChI=1S/C21H25Cl2N3O.2ClH/c1-3-26(12-9-22)11-...,24853
3,28925,"InChI=1S/C5H11Cl2N/c1-8(4-2-6)5-3-7/h2-5H2,1H3",22333
4,40910,InChI=1S/N3/c1-3-2/q-1,25355
...,...,...,...
10483,190185,InChI=1S/C32H52N4O19/c1-8-19(41)22(44)15(33-11...,59174
10484,190305,InChI=1S/C61H113N3O20/c1-5-7-9-11-13-15-16-17-...,53000
10485,190306,InChI=1S/C67H123N3O24/c1-6-8-10-12-14-16-17-18...,53000
10486,190307,InChI=1S/C56H106N2O16/c1-5-7-9-11-13-15-16-17-...,53000


WARNING: running the below code will take approximately 4 hours.

Feel free to run the below code. 

If you would like to skip this step and access the file which it produces directly, you can find it under:

'chebiId_pubchemdata.tsv'

Otherwise, it is loaded 6 cells below into a pandas dataframe.

In [11]:
# preparing an output file
fout=open('chebiId_pubchemdata.tsv','w')
fout.write("ChEBI\tMolecularFormula\tMolecularWeight\tXLogP\tExactMass\tMonoisotopicMass\tTPSA\tComplexity\tCharge\tHBondDonorCount\tHBondAcceptorCount\tRotatableBondCount\tHeavyAtomCount\tIsotopeAtomCount\tAtomStereoCount\tDefinedAtomStereoCount\tUndefinedAtomStereoCount\tBondStereoCount\n")

# cycling through the input file, prepare the request, and call PubChem's APIs. Then we write each result to a file
count=0

for i in range(len(df)):
    
    # using InChI as data to be passed to the post
    obj={'inchi':df.InChI[i]}
    
    # making the API request
    res=requests.post(url,obj)
    
    # just watching the counter grow
    count+=1
    print(str(count), end='\r')
    sys.stdout.flush()
    
    # using the json of the reponse
    rjson=res.json()
    
    # and checking if it was filled (some may be empty)
    if('PropertyTable' in rjson):
        
        if('Properties' in rjson['PropertyTable']):
            
            # getting the first (and only) element, which is our compound
            js=res.json()['PropertyTable']['Properties'][0]
 
            # here we define how to treat missing values, using "N/A" for all
            for k in ["MolecularFormula",
                      "MolecularWeight",
                      "XLogP",
                      "ExactMass",
                      "MonoisotopicMass",
                      "TPSA",
                      "Complexity",
                      "Charge",
                      "HBondDonorCount",
                      "HBondAcceptorCount",
                      "RotatableBondCount",
                      "HeavyAtomCount",
                      "IsotopeAtomCount",
                      "AtomStereoCount",
                      "DefinedAtomStereoCount",
                      "UndefinedAtomStereoCount",
                      "BondStereoCount"]:
            
                js.setdefault(k, "N/A")
                
            # writing the line into the output file, using tab as a separator, thus creating a tsv
            # some fields are numeric, so we need to convert them into strings
            fout.write(str(df.ChEBI[i])+"\t"+
                       js["MolecularFormula"]+"\t"+
                       js["MolecularWeight"]+"\t"+
                       str(js["XLogP"])+"\t"+
                       str(js["ExactMass"])+"\t"+
                       str(js["MonoisotopicMass"])+"\t"+
                       str(js["TPSA"])+"\t"+
                       str(js["Complexity"])+"\t"+
                       str(js["Charge"])+"\t"+
                       str(js["HBondDonorCount"])+"\t"+
                       str(js["HBondAcceptorCount"])+"\t"+
                       str(js["RotatableBondCount"])+"\t"+
                       str(js["HeavyAtomCount"])+"\t"+
                       str(js["IsotopeAtomCount"])+"\t"+
                       str(js["AtomStereoCount"])+"\t"+
                       str(js["DefinedAtomStereoCount"])+"\t"+
                       str(js["UndefinedAtomStereoCount"])+"\t"+
                       str(js["BondStereoCount"])+"\n")
            
            # this just refreshes the file every line so we can watch the file grow
            fout.flush()
            
fout.close()   

### Creating Dataframe

In [12]:
# load roles
roleinfodf=pd.read_csv('role_name_type.tsv', sep='\t')
print(roleinfodf.columns)

# load chebi, inchi, roles into dataframe
roledf=pd.read_csv('chebiId_inchi_role.tsv', sep='\t')
print(roledf.columns)

Index(['Role', 'Name', 'Type'], dtype='object')
Index(['ChEBI', 'InChI', 'Role'], dtype='object')


I can access Name and Type through the role

In [36]:
roleinfodf

,Role,Name,Type
0,15022,electron donor,chemical role
1,17499,hydrogen donor,chemical role
2,22153,acaricide,application
3,22180,EC 2.2.1.6 (acetolactate synthase) inhibitor,biological role
4,22333,alkylating agent,biological role
...,...,...,...
598,170010,EC 2.7.7.48 (RNA-directed RNA polymerase) inhi...,biological role
599,173084,ferroptosis inhibitor,biological role
600,173085,ferroptosis inducer,biological role
601,176497,geroprotector,application


I can tie together ChEBI with Name and Type through joining on role.
I need ChEBI because that's how I can join onto the pubchem df we will create below.

In [37]:
roledf

,ChEBI,InChI,Role
0,19032,InChI=1S/C2H4BrCl/c3-1-2-4/h1-2H2,25435
1,21182,InChI=1S/C23H28Cl3N3O.2ClH/c1-16(4-3-11-29(12-...,24853
2,21183,InChI=1S/C21H25Cl2N3O.2ClH/c1-3-26(12-9-22)11-...,24853
3,28925,"InChI=1S/C5H11Cl2N/c1-8(4-2-6)5-3-7/h2-5H2,1H3",22333
4,40910,InChI=1S/N3/c1-3-2/q-1,25355
...,...,...,...
10483,190185,InChI=1S/C32H52N4O19/c1-8-19(41)22(44)15(33-11...,59174
10484,190305,InChI=1S/C61H113N3O20/c1-5-7-9-11-13-15-16-17-...,53000
10485,190306,InChI=1S/C67H123N3O24/c1-6-8-10-12-14-16-17-18...,53000
10486,190307,InChI=1S/C56H106N2O16/c1-5-7-9-11-13-15-16-17-...,53000


This is equivalent to a SQL full outer join. 
Drop the InChI and role id since we don't need them. 

In [13]:
# By default the join is done on the only common attribute, which is ChEBI

chebiroledf = pd.merge(roledf.drop(['InChI'],axis=1), roleinfodf, how='outer').drop(['Role'],axis=1)
chebiroledf

,ChEBI,Name,Type
0,19032,mutagen,biological role
1,28741,mutagen,biological role
2,28775,mutagen,biological role
3,27666,mutagen,biological role
4,28954,mutagen,biological role
...,...,...,...
10483,39874,EC 2.7.7.48 (RNA-directed RNA polymerase) inhi...,biological role
10484,183312,EC 3.4.24.24 (gelatinase A) inhibitor,biological role
10485,184304,spin label,application
10486,184305,spin label,application


In [14]:
# load pubchem info into dataframe
pubchemdf=pd.read_csv('chebiId_pubchemdata.tsv', sep='\t')
print(pubchemdf.columns)

Index(['ChEBI', 'MolecularFormula', 'MolecularWeight', 'XLogP', 'ExactMass',
       'MonoisotopicMass', 'TPSA', 'Complexity', 'Charge', 'HBondDonorCount',
       'HBondAcceptorCount', 'RotatableBondCount', 'HeavyAtomCount',
       'IsotopeAtomCount', 'AtomStereoCount', 'DefinedAtomStereoCount',
       'UndefinedAtomStereoCount', 'BondStereoCount'],
      dtype='object')


In [38]:
pubchemdf

,ChEBI,MolecularFormula,MolecularWeight,XLogP,ExactMass,MonoisotopicMass,TPSA,Complexity,Charge,HBondDonorCount,HBondAcceptorCount,RotatableBondCount,HeavyAtomCount,IsotopeAtomCount,AtomStereoCount,DefinedAtomStereoCount,UndefinedAtomStereoCount,BondStereoCount
0,19032,C2H4BrCl,143.410,1.6,141.918490,141.918490,0.0,10.0,0.0,0.0,0.0,1.0,4.0,0.0,0,0.0,0.0,0
1,21182,C23H30Cl5N3O,541.800,NaN,541.080201,539.083151,37.4,494.0,0.0,3.0,4.0,11.0,32.0,0.0,1,0.0,1.0,0
2,21183,C21H27Cl4N3O,479.300,NaN,479.087873,477.090823,37.4,442.0,0.0,3.0,4.0,9.0,29.0,0.0,0,0.0,0.0,0
3,28925,C5H11Cl2N,156.050,0.9,155.026855,155.026855,3.2,43.0,0.0,0.0,1.0,4.0,8.0,0.0,0,0.0,0.0,0
4,40910,N3-,42.021,1.5,42.009222,42.009222,3.0,15.0,-1.0,0.0,2.0,0.0,3.0,0.0,0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10448,190185,C32H52N4O19,796.800,-6.2,796.322575,796.322575,340.0,1390.0,0.0,11.0,19.0,11.0,55.0,0.0,20,20.0,0.0,0
10449,190305,C61H113N3O20,1208.600,8.9,1207.791743,1207.791743,362.0,1780.0,0.0,13.0,20.0,47.0,84.0,0.0,17,17.0,0.0,0
10450,190306,C67H123N3O24,1354.700,8.3,1353.849652,1353.849652,421.0,2080.0,0.0,15.0,24.0,49.0,94.0,0.0,22,22.0,0.0,0
10451,190307,C56H106N2O16,1063.400,10.8,1062.754235,1062.754235,275.0,1420.0,0.0,10.0,16.0,42.0,74.0,0.0,16,16.0,0.0,0


Create the final dataset, again with a full outer join between the above created dataframe and the pubchem dataframe, leaving target variable at the end

In [15]:
# reordering the role columns, so the role name is the last column
# renaming the role columns to a less ambiguous version of their namesb
# replacing html codes for greek letters alpha, beta, gamma, with their English names

datasetdf = pd.merge(pubchemdf,
                     chebiroledf
                     .reindex(columns=['ChEBI','Type','Name'])
                     .rename(columns={"Name":"RoleName","Type":"RoleType"})
                     .replace(regex=['&#945;'], value='alpha')
                     .replace(regex=['&#946;'], value='beta')
                     .replace(regex=['&#947;'], value='gamma'),
                     how='outer')

datasetdf

,ChEBI,MolecularFormula,MolecularWeight,XLogP,ExactMass,MonoisotopicMass,TPSA,Complexity,Charge,HBondDonorCount,HBondAcceptorCount,RotatableBondCount,HeavyAtomCount,IsotopeAtomCount,AtomStereoCount,DefinedAtomStereoCount,UndefinedAtomStereoCount,BondStereoCount,RoleType,RoleName
0,19032,C2H4BrCl,143.410,1.6,141.918490,141.918490,0.0,10.0,0.0,0.0,0.0,1.0,4.0,0.0,0.0,0.0,0.0,0.0,biological role,mutagen
1,21182,C23H30Cl5N3O,541.800,NaN,541.080201,539.083151,37.4,494.0,0.0,3.0,4.0,11.0,32.0,0.0,1.0,0.0,1.0,0.0,biological role,intercalator
2,21183,C21H27Cl4N3O,479.300,NaN,479.087873,477.090823,37.4,442.0,0.0,3.0,4.0,9.0,29.0,0.0,0.0,0.0,0.0,0.0,biological role,intercalator
3,28925,C5H11Cl2N,156.050,0.9,155.026855,155.026855,3.2,43.0,0.0,0.0,1.0,4.0,8.0,0.0,0.0,0.0,0.0,0.0,biological role,alkylating agent
4,40910,N3-,42.021,1.5,42.009222,42.009222,3.0,15.0,-1.0,0.0,2.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,biological role,mitochondrial respiratory-chain inhibitor
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10483,131528,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,biological role,bacterial metabolite
10484,90785,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,biological role,bacterial metabolite
10485,77774,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,biological role,phosphodiesterase IV inhibitor
10486,83335,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,application,antifungal agrochemical


In [16]:
# writing the dataframe to tsv file
datasetdf.to_csv("dataset.tsv", sep="\t")